In [ ]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from collections import defaultdict
from sklearn.metrics import accuracy_score
import random

In [ ]:
df=pd.read_csv("temp_data3.csv")
df=df.iloc[:4000,:]

In [ ]:
data=df["cleaned_text"].apply(lambda x: " ".join(x.lower() for x in x.split() if x.isalpha()))
data=data.str.replace("[^\w\s]","")#harf dışı simgeler,noktalama
data= data.str.replace(" \d+", " ")#sayılar 
nltk.download("stopwords")
duraklama_kelimeleri = stopwords.words()
data=pd.DataFrame(data,columns=["cleaned_text"])

In [ ]:
cleaned_text=list(data["cleaned_text"])
name=["good","love","some"]
data_label=[]
sayac=0
for sentences in cleaned_text:
    for word in sentences.split():
        if word not in name:
            sayac=3
        elif(word==name[0]):
            sayac=0
            break
        elif(word==name[1]):
            sayac=1
            break
        elif(word==name[2]):
            sayac=2
            break
    if(sayac==0):
        data_label.append(0)
    elif(sayac==1):
        data_label.append(1)
    elif(sayac==2):
        data_label.append(2)
    elif(sayac==3):
        data_label.append(3)

In [ ]:
data_label=pd.DataFrame(data_label,columns=["Labels"])
dataf=pd.concat([data,data_label],axis=1)
dataf

In [ ]:
list1=[]
list1=dataf.Labels.values
for i in range(len(list1)):
    if(list1[i]==3):
        dataf.drop([i],axis=0,inplace=True)

In [ ]:
cleaned_text=list(dataf["cleaned_text"])
dataf

In [ ]:
np.sum(dataf["Labels"]==0),np.sum(dataf["Labels"]==1),np.sum(dataf["Labels"]==2),

In [ ]:
def IDF(corpus, unique_words):
    idf_dict={}
    N=len(corpus)
    for i in unique_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1#kelimenin kaç dökümanda olduğu
            idf_dict[i]=(math.log((1+N)/(count+1)))+1
    return idf_dict 

In [ ]:
def fit(whole_data):
    unique_words = set()
    if isinstance(whole_data, (list,)):
        for x in whole_data:
            for y in x.split():#cümle kelimeleri
                if len(y)<2:
                    continue
                unique_words.add(y)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}#kelime indexleme
    Idf_values_of_all_unique_words=IDF(whole_data,unique_words)
    return vocab, Idf_values_of_all_unique_words
Vocabulary, idf_of_vocabulary=fit(cleaned_text) 

In [ ]:
def transform(dataset,vocabulary,idf_values):
    tfidf=np.zeros(len(vocabulary))
    number_of_words_in_sentence=Counter(dataset.split())#yorumdaki kelimelerin sayısı
    for word in dataset.split():
        if word in  list(vocabulary.keys()):#(kelimenin yorumdaki sayısı/yorumun uzunluğu)*kelimenin idf değeri
            tf_idf_value=(number_of_words_in_sentence[word]/len(dataset.split()))*(idf_values[word])
            tfidf[vocabulary[word]]=tf_idf_value
    return tfidf

In [ ]:
tablo = np.zeros((len(cleaned_text),len(Vocabulary)))
tablo =pd.DataFrame(tablo,columns=Vocabulary.keys())
tfidf=np.zeros(len(Vocabulary))
for i in range(len(cleaned_text)):
    tfidf=transform(cleaned_text[i],Vocabulary,idf_of_vocabulary)
    tablo.loc[i] = tfidf

In [ ]:
tablo=normalize(tablo, norm='l2', axis=1, copy=True, return_norm=False)
tablo =pd.DataFrame(tablo,columns=Vocabulary.keys())

In [ ]:
labels=dataf["Labels"]
y1 = np.zeros([tablo.shape[0], len(labels.unique())])
y1 = pd.DataFrame(y1)
labels.unique()

In [ ]:
labels=labels.reset_index(drop=True)

In [ ]:
for i in range(0, len(labels.unique())):
    for j in range(0, len(y1)):
        if labels[j] == labels.unique()[i]:
            y1.iloc[j, i] = 1#yorum hangi etikete sahipse 1 diğerleri 0
        else: 
            y1.iloc[j, i] = 0
y1.head()

In [ ]:
y1=y1.astype("int")

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(tablo,y1, stratify = y1)

In [ ]:
test_labels=test_labels.reset_index(drop=True)

In [ ]:
orj=np.array([]) 
for i in range(len(test_labels)):
    for j in range(len(labels.unique())):
        if(test_labels[j][i]==1):
            orj=np.append(orj,j)

In [ ]:
orj

In [ ]:
def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output

In [ ]:
init_parameters = {} 
init_parameters["weight"] = np.random.rand(train_sentences.shape[1])
init_parameters["bias"] = 0
def optimize(x, y,learning_rate,bs,iterations,parameters): 
    size = x.shape[0]#train sayısı
    weight = parameters["weight"] 
    bias = parameters["bias"]
    weight_dict = {}
    dw=[]
    
    for i in range(iterations):
        
        weight_dict[i] = defaultdict(lambda: 0)
        
        for j in range((size-1)//bs+1):#10
            start_i = j*100
            end_i = start_i + 100
            xb = x[start_i:end_i]
            yb = y[start_i:end_i]
            size2=xb.shape[0]
            sigma = sigmoid(np.dot(xb, weight) + bias)
            dW = (1/size2) * np.dot(xb.T, (sigma - yb))
            db = (1/size2) * np.sum((sigma - yb))
            weight -= learning_rate * dW
            bias -= learning_rate * db 
            
            parameters["weight"] = weight
            parameters["bias"] = bias

            weight_dict[i][j]=weight
            dw.append(dW)

    return parameters,weight_dict,dw

In [ ]:
def train(x, y, learning_rate,bs,iterations):
    parameters_out,weight_dict,dw = optimize(x, y, learning_rate, bs, iterations ,init_parameters)
    return parameters_out,weight_dict,dw

In [ ]:
parameters_out=[]
for i in range(len(labels.unique())):
    train_labels1=np.array(train_labels[i])
    parameters_out1,weight_dict,dw= train(train_sentences, train_labels1,learning_rate = 0.02,bs=100,iterations = 200)
    parameters_out.append(parameters_out1)

In [ ]:
dw # her 10 batch'ten sonra yeni epoch

In [ ]:
output_values_te=[]
for i in range(len(labels.unique())):
    output_values1 = np.dot(test_sentences, parameters_out[i]["weight"]) + parameters_out[i]["bias"]
    output_values_te.append(output_values1)

In [ ]:
weight_dict

In [ ]:
output_values_te

In [ ]:
pred=np.array([]) 
for i in range(len(test_sentences)):
    for j in range(len(labels.unique())):
        list1=np.array([]) 
        list1=np.append(list1,output_values_te[j][i])
    pred=np.append(pred,np.argmax(list1))

In [ ]:
accuracy_score(orj,pred)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

class_names=["0","1","2"]

print(classification_report(orj, pred, target_names=class_names))

print(confusion_matrix(orj, pred))